This program trains an artificial neural network to perform magnetic field triangulation based real experimental data.

In [27]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Load the data

In [28]:
# specify the path to the h5 file
path_to_file = 'all_data.h5' # Location of the h5 files

f = h5py.File(path_to_file, 'r')

# convert the h5 contents to a dataframe
data = {group: f[group][()] for group in f}
df = pd.DataFrame(data)

# Create training sets

In [29]:
V0 = df['V0'] # Targets
X = df.iloc[:,1:] # Features

from sklearn.model_selection import train_test_split

# split X and V0 into training and testing sets
X_train, X_test, V0_train, V0_test = train_test_split(X, V0, test_size=0.4, random_state=42)

# split X_test and V0_test into training and testing sets
X_val, X_test, V0_val, V0_test = train_test_split(X_test, V0_test, test_size=0.5, random_state=42)
